### A starting notebook for 1D diffusion

The next line activates the environment in which we are working on HW3. It is assumed that you already generated this environment, per the instructions in the document *UsingJulia.pdf*

In [ ]:
using Pkg
Pkg.activate("/Users/jeff/Dropbox/MAE250H/julia/HW3")

Load some useful packages, including our own!

In [ ]:
using Revise
using Plots
using HW3

### Problem statement

The goal of this problem is to solve

$$ \dfrac{\partial u}{\partial t} = \nu \dfrac{\partial^2 u}{\partial x^2}$$

on domain $x \in [0,L]$ subject to boundary conditions $u(0,t) = u_L$ and $u(L,t) = u_R$ and initial condition $u(x,0) = 0$.

### Set up the problem

In [ ]:
L = 1  # length of domain
xL = 0.0  # coordinate of left side of domain

Set physical parameters of the problem

In [ ]:
ν = 1
uL = 1
uR = 2

Set the spatial discretization and the mapping to physical space

In [ ]:
N = 100

In [ ]:
Δx = L/N
# Note that these are functions, not arrays.
xedge(i_e::Real) = xL + (i_e - 1)*Δx
xcell(i_c::Real) = xL + (i_c - 1.5)*Δx
i_c_int = 2:N+1 # interior cells
i_e = 1:N+1 # edges

Here we will define functions for use in integrating the problem

In [ ]:
# RHS function of diffusion problem for edge data
"""
        diffuse1d_dirichlet(t,u::EdgeData1D) -> EdgeData1D

Computes the right-hand side of the 1-d diffusion problem at the current time `t` and state (1-d edge data `u`).
"""
function diffuse1d_dirichlet(u::HW3.EdgeData1D{N},t::Real) where {N}
    du = HW3.EdgeData1D(u)
    
    # set boundary values to Dirichlet values before applying Laplacian
    u[1] = uL
    u[N+1] = uR
    
    du = HW3.laplacian(u)
    
    return ν/Δx^2*du
end

#====== you write the following =====#

# RHS function of diffusion problem for cell-centered data
"""
        diffuse1d_dirichlet(t,u::CellData1D) -> CellData1D

Computes the right-hand side of the 1-d diffusion problem at the current time `t` and state (1-d cell data `u`).
"""
function diffuse1d_dirichlet(u::HW3.CellData1D{N},t::Real) where {N}
    du = HW3.CellData1D(u)
    
    # write this
    
    return ν/Δx^2*du
end



# Crank-Nicolson for edge-based data
"""
        diffuse1d_dirichlet_cn(t,u::EdgeData1D) -> Real, EdgeData1D

Advances the state vector (1-d edge data `u`) by one time step and returns the new time
and the value of `u` at that time.
"""
function diffuse1d_dirichlet_cn(t::Real,u::HW3.EdgeData1D{N}) where {N}
    unp1 = deepcopy(u)
    
    # write this
    
    return t+Δt, unp1
    
end

# Crank-Nicolson for cell-centered data
"""
        diffuse1d_dirichlet_cn(t,u::CellData1D) -> Real, CellData1D

Advances the state vector (1-d cell data `u`) by one time step and returns the new time
and the value of `u` at that time.
"""
function diffuse1d_dirichlet_cn(t::Real,u::HW3.CellData1D{N}) where {N}
    unp1 = deepcopy(u) # currently hold uⁿ    
    
    # write this
    
    return t+Δt, unp1
    
end

#### Edge-based solution, using RK4

Initial condition

In [ ]:
u0 = HW3.EdgeData1D(N);

Steady-state exact solution

In [ ]:
uss_edge = HW3.EdgeData1D(u0)
uss_edge[i_e] .= (uR-uL)/L*xedge.(i_e) .+ uL;

Set up time step and integrator

In [ ]:
Fo = 0.5 # Fourier number: this choice should be explored!
Δt = Fo*Δx^2/ν 
rk = HW3.RK(u0,Δt,diffuse1d_dirichlet;rk=HW3.RK4)

### Solve the problem

Integrate this to final time of tf. Try different final times to explore the evolution of the problem.

In [ ]:
tf = 0.1
nsteps = ceil(Int,tf/Δt)

# Initial conditions
t = 0.0
u = deepcopy(u0)

# Set up some arrays to store data
uarray = [deepcopy(u)]
tarray = [t]

# March!
for j = 1:nsteps
    t, u = rk(t,u)
    
    # enforce boundary conditions on result
    u[1] = uL
    u[N+1] = uR
    
    push!(uarray,deepcopy(u))
    push!(tarray,t)
end

Plot the result and compare with steady-state exact solution

In [ ]:
plot(xedge.(i_e),uarray[end][i_e],xlim=(0,1),ylim=(0,2),label=string("numerical at t=",tarray[end]),xlabel="x")
plot!(xedge.(i_e),uss_edge,label="steady-state exact")

#### Try it with cell-centered data and Dirichlet conditions

#### Use Crank-Nicolson time marching method to solve on edges or cells